In [2]:
from myutility import descriptor_selection

descriptor_selection

===== 'Tg' 예측을 위한 상위 20개 디스크립터 =====
FractionCSP3         0.402691
SlogP_VSA8           0.143457
VSA_EState7          0.034622
VSA_EState4          0.022283
VSA_EState1          0.021426
fr_unbrch_alkane     0.018064
NumRotatableBonds    0.016869
VSA_EState8          0.015656
Kappa3               0.010463
FpDensityMorgan3     0.010316
BalabanJ             0.010027
MinAbsEStateIndex    0.009170
VSA_EState5          0.008839
EState_VSA8          0.008552
EState_VSA2          0.008346
qed                  0.008262
SlogP_VSA1           0.008129
SMR_VSA10            0.007472
MolLogP              0.007413
SlogP_VSA10          0.007362
dtype: float64

피처 중요도 그래프가 'feature_importance_Tg.png'으로 저장되었습니다.


<module 'myutility.descriptor_selection' from 'E:\\SSL-polyGNN\\myutility\\descriptor_selection.py'>

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# --- 이전 단계들이 완료되었다고 가정합니다 ---
# 1. 원본 데이터프레임 (df_final)
# 2. 모든 디스크립터가 계산된 데이터프레임 (df_descriptors)
# 3. 최종 선택된 디스크립터 리스트 (final_features)

# 예시 데이터 생성 (실제로는 이 코드가 필요 없습니다)
smiles_list = ['CCO', 'CCC', 'c1ccccc1', 'CNC', 'CC(=O)O', 'CCN(CC)CC']
df_final = pd.DataFrame({
    'SMILES': smiles_list * 10,
    'Tg': np.random.rand(60) * 100 + 300,
    'Tc': np.random.rand(60) * 50 + 500,
})
df_final.loc[::3, 'Tg'] = np.nan # Tg에 일부러 결측치 생성
df_final.loc[::4, 'Tc'] = np.nan # Tc에 일부러 결측치 생성

# 예시 디스크립터와 최종 피처 리스트
final_features = ['HeavyAtomCount', 'TPSA', 'FpDensityMorgan2', 'HallKierAlpha']
df_descriptors = pd.DataFrame(np.random.rand(60, len(final_features)), columns=final_features)

#-----------------------------------------------------------

# 1. Imputation에 사용할 데이터 준비
#    물성치와 최종 선택된 디스크립터를 하나의 데이터프레임으로 합칩니다.
properties_to_impute = ['Tg', 'Tc'] # 결측치를 채울 물성치 컬럼들
data_for_imputation = pd.concat([df_final[properties_to_impute], df_descriptors[final_features]], axis=1)

print("===== Imputation 이전 결측치 개수 =====")
print(data_for_imputation.isnull().sum())

# 2. Imputer 초기화
#    예측 모델로 RandomForestRegressor를 사용합니다.
#    n_estimators는 예시를 위해 작게 설정했으며, 실제로는 50~100 정도를 사용해도 좋습니다.
imputer = IterativeImputer(
    estimator=RandomForestRegressor(n_estimators=10, random_state=42),
    max_iter=5,       # 반복 횟수
    random_state=42
)

# 3. Imputation 실행 (Fit and Transform)
#    이 과정은 다소 시간이 소요될 수 있습니다.
print("\nImputation을 시작합니다...")
imputed_data_np = imputer.fit_transform(data_for_imputation)
print("Imputation 완료!")

# 4. 결과를 다시 DataFrame으로 변환
df_imputed_subset = pd.DataFrame(imputed_data_np, columns=data_for_imputation.columns)

# 5. 원본 데이터프레임 업데이트
df_imputed = df_final.copy()
for col in properties_to_impute:
    df_imputed[col] = df_imputed_subset[col]

print("\n===== Imputation 이후 결측치 개수 =====")
print(df_imputed.isnull().sum())

print("\n===== Imputation 결과 (상위 5개 행) =====")
print(df_imputed.head())

상관관계 쌍 발견: ('MinPartialCharge', 'BCUT2D_LOGPLOW'), 상관계수: 1.00
 -> 'BCUT2D_LOGPLOW' 제거 (평균 상관계수 더 높음: 0.16 > 0.16)

상관관계 쌍 발견: ('Kappa3', 'Kappa1'), 상관계수: 0.99
 -> 'Kappa3' 제거 (평균 상관계수 더 높음: 0.16 > 0.16)

제거될 디스크립터:
['Kappa3', 'BCUT2D_LOGPLOW']

최종 선택된 디스크립터:
['MinPartialCharge', 'EState_VSA8', 'MaxEStateIndex', 'MinAbsPartialCharge', 'FractionCSP3', 'Kappa1', 'SMR_VSA7', 'BCUT2D_MRHI', 'HeavyAtomCount', 'TPSA', 'FpDensityMorgan2', 'HallKierAlpha', 'SMR_VSA5', 'MinEStateIndex', 'NumHeteroatoms', 'HeavyAtomMolWt', 'BCUT2D_CHGLO', 'SlogP_VSA5']

총 20개에서 2개가 제거되어 18개가 선택되었습니다.


['MinPartialCharge', 'EState_VSA8', 'MaxEStateIndex', 'MinAbsPartialCharge', 'FractionCSP3', 'Kappa1', 'SMR_VSA7', 'BCUT2D_MRHI', 'HeavyAtomCount', 'TPSA', 'FpDensityMorgan2', 'HallKierAlpha', 'SMR_VSA5', 'MinEStateIndex', 'NumHeteroatoms', 'HeavyAtomMolWt', 'BCUT2D_CHGLO', 'SlogP_VSA5']
